# Chapter 7

Many researchers choose to do ad-hoc processing of data from one form to another using a general-purpose programming language, like `Python`, `Perl`, `R`, or `Java` or `Unix` text-processing tools. Pandas, along with Python's built-in features provide a flexible and fast high level set of tools to manipulate data.
## Handling Missing Data
Missing data is a common occurence in many data analysis applications. One of the goals of pandas is to make working with missing data as painless as possible. For example, all of the descriptive statistics on pandas objects exclude missing data by default.

The way missing data is represeneted in pandas objects is somewhat imperfect, but it is functional for most users. For numeric data, pandas uses the `**float**` value NaN (Not a Number) to represent missing data. We call this a *sentinel value* that can be easily detected:

In [1]:
import pandas as pd
import numpy as np

In [2]:
string_data = pd.Series(['aardvark' ,'artichoke', np.nan, 'avocado'])
string_data

0     aardvark
1    artichoke
2          NaN
3      avocado
dtype: object

In [3]:
string_data.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In Pandas, it is convention to refere to missing data as NA or *not available*. In statistics applicaitons, NA data may either be data that does not exist or that exists but was not observed (through problems with data collection or others). When cleaning up data for analysis, it is often important to do analysis on the missing data itself to indentify data collection problems or potential biases in the data caused by missing data.

The built in Python value `None` is also treated as NA in obkect arrays

In [4]:
string_data[0] = None
string_data.isnull()

0     True
1    False
2     True
3    False
dtype: bool

user API functions often abstract away most of the annoyig details regarding working with missing data or NA/NaN values. 

| Argument | Description |
| -------- | ----------- | 
| `dropna` | Filter axis labels based on whether values for each label have missing data, with varying thresholds for how much missing data to tolerate. |
| `fillna` | Fill in missing data with some value or using an interpolation method such as `ffill` or `bfill`. |
| `isnull` | Return boolean values indicating which values are missing/NA. | 
| `notnull` | Negation of `isnull` | 

### Filtering Out Missing Data

There are a few methods available to filer out missing data. While you can do it by hand with `pandas.isnull` and boolean indexing, the `dropna` method is also helpful. When used on a Series, it returns the Series with only the non-null data and index values:

In [8]:
from numpy import nan as NA
data = pd.Series([1, NA, 3.5, NA, 7])

In [9]:
data.dropna()
## is equivalent to 
# data[data.notnullI()]

0    1.0
2    3.5
4    7.0
dtype: float64

With DataFrame object, things are a bit more complex. You may want to drop rows or columns that are all NA or only those containing any NAs. `dropna` by default drops any row containing a missing value:

In [10]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA],
                    [NA, NA, NA], [NA, 6.5, 3.]])

In [11]:
cleaned = data.dropna()
data

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


In [12]:
cleaned

,0,1,2
0,1.0,6.5,3.0


In [13]:
# Passing how='all' will only drop rows that are all nna
data.dropna(how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
3,NaN,6.5,3.0


In [14]:
#To drop columns in the same way, pass axis=1:
data[4] = NA
data

,0,1,2,4
0,1.0,6.5,3.0,NaN
1,1.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,6.5,3.0,NaN


In [15]:
data.dropna(axis=1, how='all')

,0,1,2
0,1.0,6.5,3.0
1,1.0,NaN,NaN
2,NaN,NaN,NaN
3,NaN,6.5,3.0


A related way to filter out DataFrame rows tends to concern time series data. Suppose you want to keep only rows containing a certain number of observations. You can indicate this with the `thresh` argument:

In [16]:
df = pd.DataFrame(np.random.randn(7, 3))

In [17]:
df.iloc[:4, 1] = NA

In [18]:
df.iloc[:2, 2] = NA
df

,0,1,2
0,-0.649288,NaN,NaN
1,-1.206831,NaN,NaN
2,0.179629,NaN,-0.325133
3,-1.262227,NaN,0.562146
4,2.297852,0.148638,0.501008
5,-1.487410,-0.323249,-0.597056
6,-0.230135,1.099626,1.058672


In [19]:
df.dropna()

,0,1,2
4,2.297852,0.148638,0.501008
5,-1.487410,-0.323249,-0.597056
6,-0.230135,1.099626,1.058672


In [20]:
df.dropna(thresh=2)

,0,1,2
2,0.179629,NaN,-0.325133
3,-1.262227,NaN,0.562146
4,2.297852,0.148638,0.501008
5,-1.487410,-0.323249,-0.597056
6,-0.230135,1.099626,1.058672


### Filling in Missing Data 
Rather than filtering out missing data (and potentially discarding other data along with it), you may want to fill in the "holes in any number of ways. For most purposes, the `fillna` method is the workhorse function to use. Calling `fillna` with a constant replaces missing values with that value:

In [21]:
df.fillna(0)

,0,1,2
0,-0.649288,0.000000,0.000000
1,-1.206831,0.000000,0.000000
2,0.179629,0.000000,-0.325133
3,-1.262227,0.000000,0.562146
4,2.297852,0.148638,0.501008
5,-1.487410,-0.323249,-0.597056
6,-0.230135,1.099626,1.058672


In [22]:
# Calling fillna with a dict, you can use a different fill value for each column
df.fillna({1: 0.5, 2: 0})
# fillna returns a new object, but you can modify the existing object in-place:

,0,1,2
0,-0.649288,0.500000,0.000000
1,-1.206831,0.500000,0.000000
2,0.179629,0.500000,-0.325133
3,-1.262227,0.500000,0.562146
4,2.297852,0.148638,0.501008
5,-1.487410,-0.323249,-0.597056
6,-0.230135,1.099626,1.058672


In [23]:
df.fillna(0, inplace=True) # would not return output of new object
df

,0,1,2
0,-0.649288,0.000000,0.000000
1,-1.206831,0.000000,0.000000
2,0.179629,0.000000,-0.325133
3,-1.262227,0.000000,0.562146
4,2.297852,0.148638,0.501008
5,-1.487410,-0.323249,-0.597056
6,-0.230135,1.099626,1.058672


In [25]:
# The same interpolation methods available for reindexing can be used with fillna:
df = pd.DataFrame(np.random.randn(6, 3))

In [28]:
df.iloc[2:, 1] = NA
df.iloc[4:, 2] = NA
df

,0,1,2
0,1.586022,0.565048,-0.392873
1,-1.763886,-1.157768,0.042191
2,-0.481307,NaN,0.474447
3,-2.254965,NaN,-1.674443
4,0.139516,NaN,NaN
5,1.034733,NaN,NaN


In [29]:
df.fillna(method='ffill')

,0,1,2
0,1.586022,0.565048,-0.392873
1,-1.763886,-1.157768,0.042191
2,-0.481307,-1.157768,0.474447
3,-2.254965,-1.157768,-1.674443
4,0.139516,-1.157768,-1.674443
5,1.034733,-1.157768,-1.674443


In [30]:
df.fillna(method='ffill', limit=2)

,0,1,2
0,1.586022,0.565048,-0.392873
1,-1.763886,-1.157768,0.042191
2,-0.481307,-1.157768,0.474447
3,-2.254965,-1.157768,-1.674443
4,0.139516,NaN,-1.674443
5,1.034733,NaN,-1.674443


With `fillna` you can do lots of other things with a little creativity. For exmaple, you might pass the mean or the median value of a Series:

In [32]:
data = pd.Series([1., NA, 3.5, NA, 7])

In [33]:
data.fillna(data.mean())

0    1.000000
1    3.833333
2    3.500000
3    3.833333
4    7.000000
dtype: float64

| Argument | Description |
| -------- | ----------- | 
| `value` | Scalar value or dict-like object to use to fill missing values | 
| `method` | Interpolation; by default'ffill'if function called with no other arguments | 
| `axis` | Axis to fill on; default axis=0 kinda like column or row | 
| `inplace` | Modify the calling object without producing a copy | 
| `limit` | For forward and backward filling, maximum number of consevutive periods to fill | 

# Data Transformation
## Removing Duplicates
Duplicate rows may be found in a DataFrame for any number of reasons. Here is an example:

In [34]:
data = pd.DataFrame({'k1': ['one', 'two'] * 3 + ['two'],
                    'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4
6,two,4


In [35]:
# The DataFrame method `duplicated` returns a boolean Series indicating whether each row is a duplicated value 
# This is determined if the row has been observed in a previous row
data.duplicated()

0    False
1    False
2    False
3    False
4    False
5    False
6     True
dtype: bool

Relatedly, `drop_duplicates` returns a DataFrame where the duplicated array is `False`:

In [36]:
data.drop_duplicates()

,k1,k2
0,one,1
1,two,1
2,one,2
3,two,3
4,one,3
5,two,4


Both of these methods by default consider all of the columnsl alternatively, you can specify any subset of them to detect duplicates. Suppose we had an additional column of values and wanted to filter duplicate only based on the 'k1' column:

In [37]:
data['v1'] = range(7)

In [38]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
1,two,1,1


`duplicated` and `drop_duplicates` by defualt keep the first observed value combinaion. Passing `keep='last'` will return the last one:

In [39]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
0,one,1,0
1,two,1,1
2,one,2,2
3,two,3,3
4,one,3,4
6,two,4,6


## Transforming Data Using a Function or Mapping
For many datasets, you may wish to preform some transformation based on the values in an array, Series, or column in a DataFrame. 

In [40]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon',
                              'Pastrami', 'corned beef', 'Bacon',
                              'pastrami', 'honey ham', 'nova lox'],
                    'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


Supose you wanted to add a column indicating the type of animal that each food came from. Let's write down a mapping of each distinct meat type to the kind of animal:

In [44]:
meat_to_animal = {
    'bacon': 'pig',
    'pulled pork': 'pig',
    'pastrami': 'cow',
    'corned beef': 'cow',
    'honey ham': 'pig',
    'nova lox': 'salmon'
}

The map method on a Series accepts a function or dict-like object containing a map‐ ping, but here we have a small problem in that some of the meats are capitalized and others are not. Thus, we need to convert each value to lowercase using the str.lower Series method:

In [45]:
lowercased = data['food'].str.lower()
lowercased

0          bacon
1    pulled pork
2          bacon
3       pastrami
4    corned beef
5          bacon
6       pastrami
7      honey ham
8       nova lox
Name: food, dtype: object

In [46]:
data['animal'] = lowercased.map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


You could have also passed a function that does all the work:

In [47]:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

Using `map` is a convienient way to preform element-wise transformations and other data cleaning-related operations
## Replacing Values
Filling in missing data with the `fillna` method is a special case of more general value replacement. As you’ve already seen, map can be used to modify a subset of values in an object but replace provides a simpler and more flexible way to do so. Let’s con‐sider this Series:

In [48]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The `-999` values might be sentinel values for missing data. To replace these with Na values that pandas understands, we can use `replace`, producing a new Series (unless you pass `inplace=True`):

In [49]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [51]:
# If you want to replace mutliple values at once, you instead pass a list and thne the substitute value:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [52]:
# To use a different replacement for each value, pass a list of substitues:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [53]:
#The argument bassed can also be a dict!
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

The `data.replace` method is distinct form `data.str.replace`, which preforms a string substitution element-wise. 
## Renaming Axis Indexes
Like values in a Series, axis labels can be similarly transformed by a function or map‐ ping of some form to produce new, differently labeled objects. You can also modify the axes in-place without creating a new data structure. Here’s a simple example:

In [54]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)),
                   index=['Ohio', 'Colorado', 'New York'],
                   columns=['one', 'two', 'three', 'four'])

In [55]:
# Like a Series, the axis index have a map method:
transform = lambda x: x[:4].upper()

In [56]:
data.index.map(transform)

Index(['OHIO', 'COLO', 'NEW '], dtype='object')

In [58]:
# You can assign to `index`, modifying the Dataframe, in-place:
data.index = data.index.map(transform)
data

,one,two,three,four
OHIO,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


In [59]:
# To create a transformed version of a dataset without modifyng the original, a useful method is `rename`:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colo,4,5,6,7
New,8,9,10,11


In [60]:
# Notably, `rename` can be used in conjuction with a dict-like object providing new values for a subset of the axis labels
data.rename(index={'OHIO': 'INDIANA'},
           columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


`rename` saves you form the chore of copying the DataFrame manually and assigning to its `index` and `columns` attributes. Should you wish to modify a dataset in-place, pass `inplace=True`:

In [61]:
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLO,4,5,6,7
NEW,8,9,10,11


## Discretization and Binning
Continuous data is often discretized or otherwise separated into “bins” for analysis. Suppose you have data about a group of people in a study, and you want to group them into discrete age buckets:

In [62]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [63]:
# lets divide The ages int obins of 18-25, 26-35, 36-60, and 61 to older. For this, use pandas.cut():
bins = [18, 25, 35, 60, 100]

In [64]:
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64, right]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object `pandas` returns is a special `Categorical` object. The output you see describes the bins computed by `pandas.cut`. You can treat it like an array of strings indicating the bin name; internally it contains a `categories` array specifying the dis‐ tinct category names along with a labeling for the `ages` data in the codes attribute:

In [66]:
cats.codes

array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

In [67]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]], dtype='interval[int64, right]')

In [68]:
pd.value_counts(cats)

(18, 25]     5
(25, 35]     3
(35, 60]     3
(60, 100]    1
dtype: int64

Note that `pd.value_counts(cats)` are the bin counts for the result of `pandas.cut`
Consistent with mathematical notation for intervals, a parenthesis means that the side is *open*, while the square bracket means it is *closed* (inclusive). You can change which side is closed by passing `right=False`:

In [69]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64, left]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [70]:
# You can also pass you rown bin names by passing a list or array to the labels opton:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [71]:
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'YoungAdult', 'Youth', ..., 'YoungAdult', 'Senior', 'MiddleAged', 'MiddleAged', 'YoungAdult']
Length: 12
Categories (4, object): ['Youth' < 'YoungAdult' < 'MiddleAged' < 'Senior']

If you pass an integer number of bins to `cut` instead of explicit bin edges, it will compute equal-length bins based on the minimum and maximum values in the data. Consider the case of some uniformly distributed data chopped into fourths:

In [72]:
data = np.random.rand(20)

In [73]:
pd.cut(data, 4, precision=2)

[(0.24, 0.46], (0.68, 0.9], (0.46, 0.68], (0.02, 0.24], (0.02, 0.24], ..., (0.24, 0.46], (0.24, 0.46], (0.24, 0.46], (0.02, 0.24], (0.24, 0.46]]
Length: 20
Categories (4, interval[float64, right]): [(0.02, 0.24] < (0.24, 0.46] < (0.46, 0.68] < (0.68, 0.9]]

The precision=2 option limits the decimal precision to two digits.

A closely related function, `qcut`, bins the data based on sample quantiles. Depending on the distribution of the data, using `cut` will not usually result in each bin having the same number of data points. Since `qcut` uses sample quantiles instead, by definition you will obtain roughly equal-size bins:

In [75]:
data = np.random.randn(1000) # Normally distributed

In [76]:
cats = pd.qcut(data, 4) # Cut in quartiles
cats

[(0.605, 3.827], (-3.155, -0.675], (-3.155, -0.675], (-0.0217, 0.605], (-3.155, -0.675], ..., (0.605, 3.827], (-3.155, -0.675], (-3.155, -0.675], (-0.0217, 0.605], (-0.675, -0.0217]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.155, -0.675] < (-0.675, -0.0217] < (-0.0217, 0.605] < (0.605, 3.827]]

In [78]:
pd.value_counts(cats)

(-3.155, -0.675]     250
(-0.675, -0.0217]    250
(-0.0217, 0.605]     250
(0.605, 3.827]       250
dtype: int64

In [80]:
# Similar to `cut` you can pass your own quantiles (between 0 and 1, inclusive)
pd.qcut(data, [0, 0.1, 0.5, 0.8, 1.])

[(0.722, 3.827], (-3.155, -1.226], (-1.226, -0.0217], (-0.0217, 0.722], (-1.226, -0.0217], ..., (0.722, 3.827], (-1.226, -0.0217], (-1.226, -0.0217], (-0.0217, 0.722], (-1.226, -0.0217]]
Length: 1000
Categories (4, interval[float64, right]): [(-3.155, -1.226] < (-1.226, -0.0217] < (-0.0217, 0.722] < (0.722, 3.827]]

We’ll return to cut and qcut later in the chapter during our discussion of aggregation and group operations, as these discretization functions are especially useful for quan‐ tile and group analysis.
## Detecting and Filtering Outliers
Filtering or transforming outliers is largely a matter of applying array operations. Consider a DataFrame with some normally distributed data:

In [82]:
data = pd.DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.003302,0.010907,-0.053197,0.014797
std,0.988908,1.063819,1.004954,1.023531
min,-3.186422,-3.955198,-3.829103,-3.386586
25%,-0.634347,-0.708640,-0.700184,-0.680013
50%,-0.001209,0.000234,-0.036576,0.006919
75%,0.662211,0.719498,0.611221,0.680994
max,3.231279,3.384508,3.639684,3.378410


In [83]:
# Suppose you want to find values in one of the columns exceeding 3 in absolute value:
col = data[2]

In [84]:
col[np.abs(col) > 3]

120   -3.062481
153   -3.093652
315    3.352784
519    3.086400
607   -3.455660
714   -3.829103
838   -3.133090
871    3.639684
Name: 2, dtype: float64

In [86]:
# To select all rows having a value exceeding 3 or -3, you can use the `any` method on a boolean DataFrame:
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
32,-3.186422,0.080256,0.547811,0.306267
71,-1.004690,-0.075105,0.007878,3.367353
120,-1.541480,0.695867,-3.062481,0.059171
153,-1.282608,0.220888,-3.093652,0.353224
212,-0.511136,3.224603,-0.972606,-1.597085
315,-0.700947,-2.182867,3.352784,0.364244
390,0.543133,3.023922,2.325101,-0.094171
425,-3.008206,1.155370,1.042896,0.581591
490,3.231279,-1.078045,-1.113831,1.189351
519,-1.109831,2.027871,3.086400,0.490102


In [87]:
# Values can be set based on these criteria. 
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.003320,0.011059,-0.052701,0.013915
std,0.987513,1.058024,0.996286,1.018408
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.634347,-0.708640,-0.700184,-0.680013
50%,-0.001209,0.000234,-0.036576,0.006919
75%,0.662211,0.719498,0.611221,0.680994
max,3.000000,3.000000,3.000000,3.000000


In [88]:
# The statement `np.sign(data) produces 1 and -1 values based on whether existing values are positive or negative:
np.sign(data).head()

,0,1,2,3
0,1.0,-1.0,-1.0,-1.0
1,1.0,-1.0,1.0,1.0
2,-1.0,1.0,1.0,1.0
3,-1.0,-1.0,-1.0,1.0
4,1.0,1.0,1.0,-1.0


## Permutation and Random Sampling
Permuting (randomnly reordering) a Series or the rows in a DataFrame is easy to do using the `numpy.random.permutation` function. Calling `permutation` with the length of the axis you want to permute produces an array of integers indicating the new ordering:

In [89]:
df = pd.DataFrame(np.arange(5 * 4).reshape((5, 4)))

In [90]:
sampler = np.random.permutation(5)
sampler

array([1, 4, 0, 3, 2])

In [91]:
# That array can then be used in `iloc` based indexing or the equivalent `take` function:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [92]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
4,16,17,18,19
0,0,1,2,3
3,12,13,14,15
2,8,9,10,11


In [93]:
# To select a random subset without replacement, you can use the `sample` method on Series and DataFrame:
df.sample(n=3)

,0,1,2,3
0,0,1,2,3
2,8,9,10,11
4,16,17,18,19


In [96]:
# To generate a sample with replacement (to allow repeat choices), pass replace=True to sample:
choices = pd.Series([5, 7, -1, 6, 4])

In [97]:
draws = choices.sample(n=10, replace=True)
draws

0    5
0    5
0    5
3    6
1    7
4    4
1    7
0    5
4    4
0    5
dtype: int64

## Computing Indicator/Dummy Values
Another type of transformation for statistical modeling or machine learning applica‐ tions is converting a categorical variable into a “dummy” or “indicator” matrix. If a column in a DataFrame has k distinct values, you would derive a matrix or Data‐ Frame with k columns containing all 1s and 0s. pandas has a get_dummies function for doing this, though devising one yourself is not difficult. Let’s return to an earlier example DataFrame:

In [98]:
df = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                  'data1': range(6)})

In [99]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In some cases, you may want to add a prefix to the columns in the indicator Data‐ Frame, which can then be merged with the other `data.get_dummies` has a prefix argu‐ment for doing this:

In [100]:
dummies = pd.get_dummies(df['key'], prefix='key')

In [102]:
df_with_dummy = df[['data1']].join(dummies)
df_with_dummy

,data1,key_a,key_b,key_c
0,0,0,1,0
1,1,0,1,0
2,2,1,0,0
3,3,0,0,1
4,4,1,0,0
5,5,0,1,0


If a row in a DataFrame belongs to multiple categories, things are a bit more compli‐ cated. Let’s look at the MovieLens 1M dataset, which is investigated in more detail in Chapter 14:

In [103]:
mnames = ['movie_id', 'title', 'genres']

In [104]:
movies = pd.read_table('datasets/movielens/movie.dat', sep='::',
                      header=None, names=mnames)

/var/folders/wm/qyykb01n2jv6nch_csqvcwxc0000gp/T/ipykernel_11727/1934810452.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_table('datasets/movielens/movie.dat', sep='::',


FileNotFoundError: [Errno 2] No such file or directory: 'datasets/movielens/movie.dat'

In [105]:
movies[:10]

NameError: name 'movies' is not defined

In [106]:
# adding indicator variables for each genre requires a little bit of wrangling. 
# First, we extract the list of unique genres in the dataset
all_genres = []

In [107]:
for x in movies.genres:
    all_genres.extend(x.plit('|'))

NameError: name 'movies' is not defined

In [108]:
genres = pd.unique(all_genres)
genres

array([], dtype=float64)

In [110]:
# One way to construct the indicator DataFrame is to start with a DataFrame of all zeros:
zero_matrix = np.zeros((len(movies), len(genres)))
dummies = pd.DataFrame(zero_matrix, columns=genres)

NameError: name 'movies' is not defined

In [111]:
gen = movies.genres[0]

NameError: name 'movies' is not defined

In [112]:
gen.split('|')

NameError: name 'gen' is not defined

In [113]:
dummies.columns.get_indexer(gen.split('|'))

NameError: name 'gen' is not defined

In [114]:
for i, gen in enumerate(movies.genres):
    indicies = dummies.columns.get_indexer(gen.split('|'))
    dummies.iloc[i, indicies] = 1

NameError: name 'movies' is not defined

In [115]:
movies_windic = movies.join(dummies.add_prefix('Genre_'))

NameError: name 'movies' is not defined

In [116]:
movies_windic.iloc[0]

NameError: name 'movies_windic' is not defined

For much larger data, this method of constructing indicator variables with multiple membership is not especially speedy. It would be better to write a lower-level function that writes directly to a NumPy array, and then wrap result in a DataFrame.
A useful recipe for statistical applications is to combine `get_dummies` with a discretization function like `cut`:

In [117]:
np.random.seed(12345)

In [118]:
values = np.random.rand(10)
values

array([0.92961609, 0.31637555, 0.18391881, 0.20456028, 0.56772503,
       0.5955447 , 0.96451452, 0.6531771 , 0.74890664, 0.65356987])

In [119]:
bin = [0, 0.2, 0.4, 0.6, 0.8, 1]

In [120]:
pd.get_dummies(pd.cut(values, bins))

,"(18, 25]","(25, 35]","(35, 60]","(60, 100]"
0,0,0,0,0
1,0,0,0,0
2,0,0,0,0
3,0,0,0,0
4,0,0,0,0
5,0,0,0,0
6,0,0,0,0
7,0,0,0,0
8,0,0,0,0
9,0,0,0,0


## 7.3 String Manipulation
Python has long been a popular raw data manipulation language in part due to its ease of use for string and text processing. Most text operations are made simple with the string object’s built-in methods. For more complex pattern matching and text manipulations, regular expressions may be needed. pandas adds to the mix by ena‐ bling you to apply string and regular expressions concisely on whole arrays of data, additionally handling the annoyance of missing data.
### String Object Methods
In many string munging and scripting applications, built-in string methods are sufficient. As an example, a comma-separated string can be broken into pieces with `split`:

In [124]:
val = 'a,b,   guido'

In [125]:
val.split(',')

['a', 'b', '   guido']

`split` is often combined with `strip` to trim whitespace (including line breaks):

In [126]:
pieces = [x.strip() for x in val.split(',')]
pieces

['a', 'b', 'guido']

In [127]:
# These substrings could be concatenated together with a two-colon delimiter using addition:
first, second, third = pieces

In [128]:
first + '::' + second + '::' + third 
# This isn't a practical generic method

'a::b::guido'

In [129]:
# A faster and more Pythonic way is to pass a list or tuple to the `join` method on the string '::'
'::'.join(pieces)

'a::b::guido'

In [132]:
# Other methods are concerned with locating substrings. 
# Using Python's `in` keyword is the best way to detect a substring,
# Though `index` and `find` can also be used:
'guido' in val

True

In [133]:
val.index(',')

1

In [134]:
val.find(':')


-1

In [135]:
# Note the difference between `find` and `index` is that `index` raises an exception if the string isnt found. 
val.index(':')

ValueError: substring not found

In [136]:
# Relatedly, `count` returns the number of occurences of a particular substring:
val.count(',')

2

In [137]:
# `replace` will substitute occurences of one pattern for another. 
# It is commonly used to delete patterns, too, by passing an empty string:
val.replace(',', '::')

'a::b::   guido'

In [138]:
val.replace(',','')

'ab   guido'

Regular expressions can also be used with many of these operations, as you'll see:

| Argument | Description | 
| -------- | ----------- | 
| `count` | Return the number of non-overlapping occurrences of substring in the string. | 
| `endswith` | Returns `True` if string ends with suffix. | 
| `startswith` | Returns `True` if string starts with prefix. |
| `join` | Use string as delimiter for concatenating a sequence of other strings. |
| `index` | Return position of first character in substring if found in the string; raises `ValueError` if not found. |
| `find` | Return position of first character of *first* occurrence of substring in the string; like `index`, but returns –1 if not found. |
| `rfind` | Return position of first character of *last* occurence of substring in the string; returns -1 if note found. |
| `replace` | Replace occurrences of string with another string. |
| `strip` | Trim whitespace, including newlines; equivalent to `x.strip()` (and `rstrip`, `lstrip`, respectively) |
| `rstrip`, `lstrip` | for each element. |
| `split` | Break string into list of substrings using passed delimiter. |
| `lower` | Convert alphabet characters to lowercase. |
| `upper` | Convert alphabet characters to uppercase. |
| `casefold` | Convert characters to lowercase, and convert any region-specific variable character combinations to a common comparable form |
| `ljust`, `rjust` | Left justify or right justify, respectively; pad opposite side of string with spaces (or some other fill character) to return a string with a minimum width |

## Regular Expressions
*Regular expressions* provide a flexible way to search or match (often more complex) string patterns in text. A single expression, commonly called a *regex*, is a string formed according to the regular expression language. Python’s built-in re module is responsible for applying regular expressions to strings; I’ll give a number of examples of its use here.

The `re` module functions fall into three categories: pattern matching, substitution, and splitting. Naturally these are all related; a regex describes a pattern to locate in the text, which can then be used for many purposes. Let’s look at a simple example:

In [139]:
import re

In [140]:
text = "foo       bar\t bas  \tqux"

In [141]:
re.split('\s+', text)

['foo', 'bar', 'bas', 'qux']

When you call `re.split('\s+', text)`, the regular expression is first *compiled*, and then its `split` method is called on the passed text. You can compile the regex yourself with `re.compile`, forming a reusable regex object:

In [143]:
regex = re.compile('\s+')

In [144]:
regex.split(text)

['foo', 'bar', 'bas', 'qux']

In [145]:
# To get a list of all the patterns matching the regex, you can use the `findall` method:
regex.findall(text)

['       ', '\t ', '  \t']

To avoid unwanted escaping with \ in regular expression, use *raw* string literals like r'C:\x' instead of the equivalent 'C:\\x'

Creating a regex object with `re.compile` is highly recommended if you intend to apply the same expression to many strings; doing so will save CPU cycles. 
`match` and `search` are closely related to `findall`. While findall returns all matches in a string, search returns only the first match. More rigidly, match *only* matches at the beginning of the string. As a less trivial example, let’s consider a block of text and a regular expression capable of identifying most email addresses:

In [146]:
text = """Dave dave@google.com
    Steve steve@gmail.com
    Rob rob@gmail.com
    Ryan ryan@yahoo.com
    """

pattern = r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,4}'
    
# re.IGNORECASE makes the regex case-insensitive
regex = re.compile(pattern, flags=re.IGNORECASE)

In [147]:
# Using `findall` on the text produces a list of email addresses:
regex.findall(text)

['dave@google.com', 'steve@gmail.com', 'rob@gmail.com', 'ryan@yahoo.com']

`search` returns a special match object for the first email address in the text. For the preceding regex, the match object can only tell us the start and end position of the pattern in the string:

In [148]:
m = regex.search(text)
m

<re.Match object; span=(5, 20), match='dave@google.com'>

In [149]:
text[m.start():m.end()]

'dave@google.com'

`regex.match` returns **None**, as it only will match if the pattern occurs at the start of the string:

In [150]:
print(regex.match(text))

None


In [151]:
print(regex.sub('REDACTED', text))

Dave REDACTED
    Steve REDACTED
    Rob REDACTED
    Ryan REDACTED
    


In [153]:
# find email addresses and simultaneously segment each address into its three components: 
# username, domain name, and domain suffix.
# To do this, put the parenthesis around the parts of the pattern to segment
pattern = r'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\.([A-Z]{2,3})'
regex = re.compile(pattern, flags=re.IGNORECASE)

In [154]:
# A match object produced by this modified regex returns a tuple of the pattern com‐ ponents with its groups method:
m = regex.match('wesm@bright.net')
m.groups()

('wesm', 'bright', 'net')

In [156]:
# `findall` returns a list of tuples when the pattern has groups:
regex.findall(text)

[('dave', 'google', 'com'),
 ('steve', 'gmail', 'com'),
 ('rob', 'gmail', 'com'),
 ('ryan', 'yahoo', 'com')]

sub also has access to groups in each match using special symbols like \1 and \2. The symbol \1 corresponds to the first matched group, \2 corresponds to the second, and so forth:

In [157]:
print(regex.sub(r'Username: \1, Domain: \2, Suffix: \3', text))

Dave Username: dave, Domain: google, Suffix: com
    Steve Username: steve, Domain: gmail, Suffix: com
    Rob Username: rob, Domain: gmail, Suffix: com
    Ryan Username: ryan, Domain: yahoo, Suffix: com
    


| Argument | Description | 
| -------- | ----------- |
| `findall` | Return all non-overlapping matching patterns in a string as a list |
| `finditer` | Like `findall`, but returns an iterator |
| `match` | Match pattern at start of string and optionally segment pattern components into groups; if the pattern
matches, returns a match object, and otherwise `None` |
| `search` | Scan string for match to pattern; returning a match object if so; unlike match, the match can be anywhere in the string as opposed to only at the beginning |
| `split` | Break string into pieces at each occurrence of pattern | 
| `sub`, `subn` | Replace all (`sub`) or first n occurrences (`subn`) of pattern in string with replacement expression; use symbols `\1, \2, ...` to refer to match group elements in the replacement string |

## Vectorized String Functions in pandas
Cleaning up a messy dataset for analysis often requires a lot of string munging and regularization. To complicate matters, a column containing strings will sometimes have missing data:

In [159]:
data =  {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com',
         'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = pd.Series(data)
data

Dave     dave@google.com
Steve    steve@gmail.com
Rob        rob@gmail.com
Wes                  NaN
dtype: object

In [160]:
data.isnull()

Dave     False
Steve    False
Rob      False
Wes       True
dtype: bool

String and regular expression methods can be applied (passing a lambda or other function) to each value using data.map, however it will fail on NaN/NA values. Series has array-oritented methods for string operations that skip NA values. Access these through Series's **`str`** attribute. For example `str.contains`:

In [161]:
data.str.contains('gmail')

Dave     False
Steve     True
Rob       True
Wes        NaN
dtype: object

In [163]:
# RegEx can also be used, along with any `re` options like `IGNORECASE`:
pattern

'([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,3})'

In [172]:
data.str.findall(pattern, flags=re.IGNORECASE)

Dave     [(dave, google, com)]
Steve    [(steve, gmail, com)]
Rob        [(rob, gmail, com)]
Wes                        NaN
dtype: object

In [170]:
# There are a variety of methods to do vectorized element removal. Either use `str.get` or index the `str` attribute
matches = data.str.match(pattern, flags=re.IGNORECASE)

In [173]:
matches

Dave     True
Steve    True
Rob      True
Wes       NaN
dtype: object

In [174]:
# To access elements in the embedded lists, we can pass an index to either of these functions:
matches.str.get(1) # depreciated method?

AttributeError: Can only use .str accessor with string values!

In [177]:
matches.str[0]

AttributeError: module 'pandas' has no attribute 'matches'

In [176]:
data.str[:5]

Dave     dave@
Steve    steve
Rob      rob@g
Wes        NaN
dtype: object

### Pandas string methods
| Method | Description |
| ------ | ----------- | 
| `cat` | Concatenate strings element-wise with optional delimiter |
| `contains` | Return boolean array if each string contains pattern/regex |
| `count` | Count occurrences of pattern |
| `extract` | Use a regular expression with groups to extract one or more strings from a Series of strings; the result will be a DataFrame with one column per group |
| `endswith` | Equivalent to `x.endswith(pattern)` for each element | 
| `startswith` | Equivalent to `x.startswith(pattern)` for each element |
| `findall` | Compute list of all occurences of pattern/regex for each string |
| `get` | Index into each element (retrieve i-th element) | 
| `isalnum` | Equivalent to built-in `str.alnum` |
| `isalpha` | Equivalent to built-in `str.isalpha` |
| `isdecimal` | Equivalent to built-in `str.isdecimal` |
| `isdigit` | Equivalent to built-in `str.isdigit` |
| `islower` | Equivalent to built-in `str.islower` |
| `isnumeric` | Equivalent to built-in `str.isnumeric` |
| `isupper` | Equivalent to built-in `str.isupper` |
| `join` | Join strings in each element of the Series with passed separator |
| `len` | Compute length of each string |
| `lower`, `upper` | Convert cases; equivalent to `x.lower()` or `x.upper()` for each element |
| `match` | Use `re.match` with the passed regular expression on each element, returning matched groups as list |
| `pad` | Add whitespace to left, right, or both sides of strings | 
| `center` | Equivalent to `pad(side='both')` |
| `repeat` | Duplicate values (e.g., `s.str.repeat(3)` is equivalent to `x * 3` for each string) |
| `replace` | Replace occurrences of pattern/regex with some other string |
| `slice` | Slice each string in the Series |
| `split` | Split strings on delimiter or regular expression |
| `strip` | Trim whitespace from both sides, including newlines |
| `rstrip` | Trim whitespace on right side | 
| `lstrip` | Trim whitespace on left side |

## Conclusion
Effective data preparation can significantly improve productive by enabling you to spend more time analyzing data and less time getting it ready for analysis. We have explored a number of tools in this chapter, but the coverage here is by no means com‐ prehensive